# GBP/USD M15 2025 — Intraday Strategy Discovery

This notebook discovers and backtests two intraday strategies on GBP/USD M15 data (2025).  
All trades open and close within the same day — **no overnight positions**.

---

## Strategy 1: Asian Range Breakout (ARB)

The 00:00–06:45 UTC window is GBP/USD's low-activity consolidation (Asian session).  
When London opens (07:00+), if price breaks outside that tight range, it often makes a sustained directional move.

| | Rule |
|---|---|
| **Signal** | 07:45 UTC close breaks above Asian High or below Asian Low |
| **Entry** | Market order at 07:45 UTC signal bar |
| **Exit** | 17:00 UTC (or SL/TP if set) |
| **Filter** | Only trade when actual breakout occurs — no breakout, no trade |

---

## Strategy 2: London Morning Momentum (LMM)

After 4 hours of London trading (08:00–12:45), GBP/USD usually has a clear trend.  
Entering at the 12:45 signal in that direction catches the continuation through the London/NY overlap.

| | Rule |
|---|---|
| **Signal** | 12:45 UTC: net move from 08:00 open exceeds `min_move_pips` |
| **Entry** | Market order at 12:45 UTC signal bar |
| **Exit** | 17:00 UTC (or SL/TP if set) |
| **Filter** | Only trade when morning move > threshold — flat days skipped |

---

**Data**: `GBP_USD_M15_20250101_20251231.csv` (~24,773 candles)

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Backtest, Strategy
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print('✓ Imports successful')

## 2. Configuration

In [ ]:
PAIR         = 'GBP_USD'
GRANULARITY  = 'M15'
FROM_DATE    = '20250101'
TO_DATE      = '20251231'

INITIAL_CASH = 10_000
COMMISSION   = 0.0001   # ~1 pip round-trip
EXIT_HOUR    = 17       # UTC hour to close all positions

DATA_DIR = Path('../data/historical')
data_path = DATA_DIR / PAIR / f'{PAIR}_{GRANULARITY}_{FROM_DATE}_{TO_DATE}.csv'

print(f'Pair       : {PAIR}')
print(f'Timeframe  : {GRANULARITY}')
print(f'Period     : {FROM_DATE} – {TO_DATE}')
print(f'Capital    : ${INITIAL_CASH:,.0f}')
print(f'Commission : {COMMISSION}')
print(f'Exit Hour  : {EXIT_HOUR}:00 UTC')
print(f'Data path  : {data_path}  {"✓" if data_path.exists() else "✗ MISSING"}')

## 3. Load Data

In [ ]:
def load_data(path: Path) -> pd.DataFrame:
    """Load OANDA CSV, return DataFrame formatted for backtesting.py."""
    df = pd.read_csv(path, comment='#', parse_dates=['time'], index_col='time')
    df.index = pd.to_datetime(df.index, utc=True)
    df.columns = [c.capitalize() for c in df.columns]
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]


df = load_data(data_path)

print(f'Candles    : {len(df):,}')
print(f'Date range : {df.index.min()} → {df.index.max()}')
print(f'Columns    : {list(df.columns)}')
df.head()

## 4. Exploratory Analysis

Understand hourly volatility and directional bias before writing strategy code.

In [ ]:
raw = df.copy().reset_index()
raw.columns = [c.lower() for c in raw.columns]
raw['hour']  = raw['time'].dt.hour
raw['range'] = (raw['high'] - raw['low']) * 10_000
raw['ret']   = (raw['close'] - raw['open']) * 10_000
raw['bull']  = raw['ret'] > 0

hourly = raw.groupby('hour').agg(
    avg_range=('range', 'mean'),
    avg_ret  =('ret',   'mean'),
    win_rate =('bull',  'mean'),
).round(2)

hrs = hourly.loc[0:22]

fig, axes = plt.subplots(1, 3, figsize=(18, 4))

# Asian session shading: 00:00–06:45
for ax in axes:
    ax.axvspan(-0.5, 6.5, alpha=0.1, color='blue', label='Asian session')
    ax.axvline(7, color='red', linestyle='--', linewidth=1.2, label='London open')
    ax.axvline(12, color='orange', linestyle='--', linewidth=1, label='LMM signal')
    ax.axvline(17, color='gray', linestyle=':', linewidth=1, label='Exit')

axes[0].bar(hrs.index, hrs['avg_range'], color='steelblue')
axes[0].set_title(f'{PAIR} — Avg Candle Range (pips)')
axes[0].set_xlabel('Hour UTC')
axes[0].legend(fontsize=7)

colors = ['green' if v > 0 else 'red' for v in hrs['avg_ret']]
axes[1].bar(hrs.index, hrs['avg_ret'], color=colors)
axes[1].axhline(0, color='black', linewidth=0.8)
axes[1].set_title(f'{PAIR} — Avg Directional Return (pips)')
axes[1].set_xlabel('Hour UTC')

axes[2].bar(hrs.index, hrs['win_rate'] * 100, color='purple', alpha=0.7)
axes[2].axhline(50, color='black', linewidth=0.8, linestyle='--')
axes[2].set_title(f'{PAIR} — Bullish Win Rate (%)')
axes[2].set_xlabel('Hour UTC')

plt.suptitle('GBP/USD M15 2025 — Hourly Statistics', fontsize=13)
plt.tight_layout()
plt.show()

# Asian session range distribution
asian_bars = raw[raw['hour'] < 7]
daily_asian_range = asian_bars.groupby(raw['time'].dt.date).agg(
    Asian_Range=('range', 'sum')
).rename_axis('date')

fig2, ax2 = plt.subplots(figsize=(8, 4))
ax2.hist(daily_asian_range['Asian_Range'], bins=40, color='steelblue', edgecolor='white', alpha=0.8)
ax2.axvline(daily_asian_range['Asian_Range'].mean(), color='red', linestyle='--',
            label=f'Mean: {daily_asian_range["Asian_Range"].mean():.1f} pips')
ax2.set_title('Distribution of Daily Asian Session Range (00:00–06:45 UTC)')
ax2.set_xlabel('Daily Asian Range (pips)')
ax2.set_ylabel('Days')
ax2.legend()
plt.tight_layout()
plt.show()

print(f'Asian session (00:00–06:45) range stats:')
print(daily_asian_range['Asian_Range'].describe().round(1))

## 5. Pre-calculate Reference Columns

Both strategies need per-day reference levels that cannot be computed inside `next()`.  
Pre-calculate and forward-fill into the M15 DataFrame (same pattern as MTF RSI strategy).

| Column | Description |
|--------|-------------|
| `Asian_High` | max(High) of bars 00:00–06:45 UTC for that day |
| `Asian_Low`  | min(Low) of bars 00:00–06:45 UTC for that day |
| `Open_0800`  | Open of the 08:00 UTC bar for that day |

In [ ]:
# Asian session high/low per day (00:00–06:45 UTC)
asian_bars = df[df.index.hour < 7]
asian = asian_bars.groupby(asian_bars.index.date).agg(
    Asian_High=('High', 'max'),
    Asian_Low=('Low', 'min')
)

# 08:00 UTC open per day
bars_0800 = df[(df.index.hour == 8) & (df.index.minute == 0)]
open_0800 = bars_0800.groupby(bars_0800.index.date)['Open'].first()

# Forward-fill into full M15 DataFrame
df['Asian_High'] = df.index.map(
    lambda t: asian.loc[t.date(), 'Asian_High'] if t.date() in asian.index else np.nan
)
df['Asian_Low'] = df.index.map(
    lambda t: asian.loc[t.date(), 'Asian_Low'] if t.date() in asian.index else np.nan
)
df['Open_0800'] = df.index.map(
    lambda t: open_0800.loc[t.date()] if t.date() in open_0800.index else np.nan
)
df.ffill(inplace=True)

# Verify
print(f'Columns: {list(df.columns)}')
nan_counts = df[['Asian_High', 'Asian_Low', 'Open_0800']].isna().sum()
print(f'NaN counts (trading hours only should be 0):')
print(nan_counts)
print()
print('Sample rows:')
sample = df[df.index.hour.isin([6, 7, 8, 12, 17])].head(15)
display(sample[['Open', 'High', 'Low', 'Close', 'Asian_High', 'Asian_Low', 'Open_0800']])

## 6. Signal Quality: Asian Range Breakout

Raw per-day analysis (no backtesting.py) — understand the base rate before coding a strategy.

In [ ]:
raw2 = df.copy().reset_index()
raw2.columns = [c.lower() for c in raw2.columns]
raw2['hour']   = raw2['time'].dt.hour
raw2['minute'] = raw2['time'].dt.minute
raw2['date']   = raw2['time'].dt.date

arb_rows = []
for date, day in raw2.groupby('date'):
    day = day.sort_values('time')
    # Signal: 07:45 UTC bar — last bar before London open
    sig = day[(day['hour'] == 7) & (day['minute'] == 45)]
    entry_bar = day[(day['hour'] == 7) & (day['minute'] == 45)]  # enter at this bar's close
    exit_bar  = day[(day['hour'] == EXIT_HOUR) & (day['minute'] == 0)]

    if len(sig) == 0 or len(exit_bar) == 0:
        continue

    sig_row    = sig.iloc[-1]
    asian_high = sig_row['asian_high']
    asian_low  = sig_row['asian_low']
    c          = sig_row['close']

    if c > asian_high:
        direction = 1   # long breakout
        signal    = 'Breakout Up'
    elif c < asian_low:
        direction = -1  # short breakout
        signal    = 'Breakout Down'
    else:
        direction = 0
        signal    = 'No Breakout'

    entry_px = c
    exit_px  = exit_bar.iloc[0]['open']
    pnl_pips = direction * (exit_px - entry_px) * 10_000 if direction != 0 else 0

    arb_rows.append({
        'date':      date,
        'dow':       pd.Timestamp(date).day_name(),
        'signal':    signal,
        'direction': direction,
        'entry_px':  entry_px,
        'exit_px':   exit_px,
        'pnl_pips':  pnl_pips,
        'win':       pnl_pips > 0 if direction != 0 else None,
        'asian_range': (asian_high - asian_low) * 10_000,
    })

arb_df = pd.DataFrame(arb_rows)

breakout_days = arb_df[arb_df['direction'] != 0]
no_signal_days = arb_df[arb_df['direction'] == 0]

print(f'Total days analysed   : {len(arb_df)}')
print(f'Days with breakout    : {len(breakout_days)} ({len(breakout_days)/len(arb_df)*100:.1f}%)')
print(f'  - Breakout Up       : {(arb_df["direction"] == 1).sum()}')
print(f'  - Breakout Down     : {(arb_df["direction"] == -1).sum()}')
print(f'Days with no breakout : {len(no_signal_days)} ({len(no_signal_days)/len(arb_df)*100:.1f}%)')
print()
print(f'When breakout detected:')
print(f'  Win rate            : {breakout_days["win"].mean()*100:.1f}%')
print(f'  Avg P&L             : {breakout_days["pnl_pips"].mean():.1f} pips')
print(f'  Total P&L           : {breakout_days["pnl_pips"].sum():.0f} pips')
print(f'  Avg Asian range     : {breakout_days["asian_range"].mean():.1f} pips')

# Cumulative P&L
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(range(len(breakout_days)), breakout_days['pnl_pips'].cumsum(), color='steelblue')
axes[0].axhline(0, color='black', linewidth=0.8, linestyle='--')
axes[0].set_title('ARB — Cumulative P&L (breakout days only, pips)')
axes[0].set_xlabel('Trade #')
axes[0].set_ylabel('Cumulative pips')

# Win rate by day of week
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
wr_dow = breakout_days.groupby('dow')['win'].mean().reindex(dow_order) * 100
axes[1].bar(wr_dow.index, wr_dow.values,
            color=['green' if v > 50 else 'red' for v in wr_dow.values])
axes[1].axhline(50, color='black', linewidth=0.8, linestyle='--')
axes[1].set_title('ARB — Win Rate by Day of Week (%)')
axes[1].set_ylabel('Win Rate (%)')
axes[1].set_xticklabels([d[:3] for d in dow_order])

plt.tight_layout()
plt.show()

## 7. Strategy 1: Asian Range Breakout — Implementation

In [ ]:
class AsianRangeBreakoutBase(Strategy):
    """
    At 07:45 UTC, if Close > Asian_High → buy; if Close < Asian_Low → sell.
    Exit at exit_hour UTC. No SL/TP — time exit only.
    """
    exit_hour = 17

    def init(self):
        pass

    def next(self):
        bar_time = self.data.index[-1]

        # Exit: close at exit_hour
        if bar_time.hour == self.exit_hour and bar_time.minute == 0 and self.position:
            self.position.close()
            return

        if self.position:
            return

        # Entry signal: 07:45 UTC
        if bar_time.hour == 7 and bar_time.minute == 45:
            c  = self.data.Close[-1]
            hi = self.data.Asian_High[-1]
            lo = self.data.Asian_Low[-1]
            if c > hi:
                self.buy()
            elif c < lo:
                self.sell()


class AsianRangeBreakout(Strategy):
    """
    Same as AsianRangeBreakoutBase but with stop loss and take profit.
    """
    exit_hour = 17
    sl_pips   = 20
    tp_pips   = 40

    def init(self):
        pass

    def next(self):
        bar_time = self.data.index[-1]

        if bar_time.hour == self.exit_hour and bar_time.minute == 0 and self.position:
            self.position.close()
            return

        if self.position:
            return

        if bar_time.hour == 7 and bar_time.minute == 45:
            c  = self.data.Close[-1]
            hi = self.data.Asian_High[-1]
            lo = self.data.Asian_Low[-1]
            pip = 0.0001
            if c > hi:
                self.buy(
                    sl=c - self.sl_pips * pip,
                    tp=c + self.tp_pips * pip
                )
            elif c < lo:
                self.sell(
                    sl=c + self.sl_pips * pip,
                    tp=c - self.tp_pips * pip
                )


print('✓ Strategy classes defined')
print('  - AsianRangeBreakoutBase : time exit only')
print('  - AsianRangeBreakout     : with SL/TP')

## 8. Backtest: Asian Range Breakout + Optimisation

In [ ]:
# Base strategy (time exit)
bt_arb_base = Backtest(df, AsianRangeBreakoutBase, cash=INITIAL_CASH, commission=COMMISSION)
stats_arb_base = bt_arb_base.run()

print('=== ARB Base (time exit) ===')
print(f"Return          : {stats_arb_base['Return [%]']:.2f}%")
print(f"Sharpe Ratio    : {stats_arb_base['Sharpe Ratio']:.3f}")
print(f"Win Rate        : {stats_arb_base['Win Rate [%]']:.1f}%")
print(f"Max Drawdown    : {stats_arb_base['Max. Drawdown [%]']:.2f}%")
print(f"# Trades        : {stats_arb_base['# Trades']}")
print()

# SL/TP strategy
bt_arb_sltp = Backtest(df, AsianRangeBreakout, cash=INITIAL_CASH, commission=COMMISSION)
stats_arb_sltp = bt_arb_sltp.run()

print('=== ARB with SL/TP (default: 20/40 pips) ===')
print(f"Return          : {stats_arb_sltp['Return [%]']:.2f}%")
print(f"Sharpe Ratio    : {stats_arb_sltp['Sharpe Ratio']:.3f}")
print(f"Win Rate        : {stats_arb_sltp['Win Rate [%]']:.1f}%")
print(f"Max Drawdown    : {stats_arb_sltp['Max. Drawdown [%]']:.2f}%")
print(f"# Trades        : {stats_arb_sltp['# Trades']}")

In [ ]:
# Optimise ARB SL/TP
print('Optimising ARB SL/TP (this may take ~1 minute)...')

bt_arb_opt = Backtest(df, AsianRangeBreakout, cash=INITIAL_CASH, commission=COMMISSION)
stats_arb_opt = bt_arb_opt.optimize(
    sl_pips=range(10, 41, 5),
    tp_pips=range(20, 81, 10),
    exit_hour=range(15, 20),
    maximize='Sharpe Ratio',
    constraint=lambda p: p.tp_pips > p.sl_pips,
    return_heatmap=False,
)

print('\n' + '=' * 60)
print('ARB OPTIMISED RESULTS')
print('=' * 60)
print(f"Return          : {stats_arb_opt['Return [%]']:.2f}%")
print(f"Sharpe Ratio    : {stats_arb_opt['Sharpe Ratio']:.3f}")
print(f"Win Rate        : {stats_arb_opt['Win Rate [%]']:.1f}%")
print(f"Max Drawdown    : {stats_arb_opt['Max. Drawdown [%]']:.2f}%")
print(f"# Trades        : {stats_arb_opt['# Trades']}")
print(f"Best SL         : {stats_arb_opt._strategy.sl_pips} pips")
print(f"Best TP         : {stats_arb_opt._strategy.tp_pips} pips")
print(f"Best exit       : {stats_arb_opt._strategy.exit_hour}:00 UTC")

In [ ]:
# Equity curves — ARB
eq_arb_base = stats_arb_base['_equity_curve']
eq_arb_sltp = stats_arb_sltp['_equity_curve']
eq_arb_opt  = stats_arb_opt['_equity_curve']

fig = go.Figure()
fig.add_trace(go.Scatter(x=eq_arb_base.index, y=eq_arb_base['Equity'],
                         name='ARB Base (time exit)', line=dict(width=1.5)))
fig.add_trace(go.Scatter(x=eq_arb_sltp.index, y=eq_arb_sltp['Equity'],
                         name='ARB SL/TP Default', line=dict(width=1.5)))
fig.add_trace(go.Scatter(x=eq_arb_opt.index, y=eq_arb_opt['Equity'],
                         name='ARB Optimised', line=dict(width=2, dash='dot')))
fig.add_hline(y=INITIAL_CASH, line_dash='dash', line_color='gray', annotation_text='Initial capital')
fig.update_layout(
    title='Asian Range Breakout — Equity Curves',
    xaxis_title='Date', yaxis_title='Equity ($)',
    height=450, hovermode='x unified'
)
fig.show()

## 9. Signal Quality: London Morning Momentum

Raw per-day analysis — 08:00→12:45 UTC move predicts 13:00→17:00 continuation.

In [ ]:
lmm_rows = []
for date, day in raw2.groupby('date'):
    day = day.sort_values('time')

    ref_bar  = day[(day['hour'] == 8) & (day['minute'] == 0)]
    sig_bar  = day[(day['hour'] == 12) & (day['minute'] == 45)]
    exit_bar = day[(day['hour'] == EXIT_HOUR) & (day['minute'] == 0)]

    if len(ref_bar) == 0 or len(sig_bar) == 0 or len(exit_bar) == 0:
        continue

    ref_px   = ref_bar.iloc[0]['open']
    sig_close = sig_bar.iloc[-1]['close']
    exit_px  = exit_bar.iloc[0]['open']
    morning_move = (sig_close - ref_px) * 10_000   # pips

    direction = 1 if morning_move > 0 else (-1 if morning_move < 0 else 0)
    pnl_pips  = direction * (exit_px - sig_close) * 10_000 if direction != 0 else 0

    lmm_rows.append({
        'date':         date,
        'dow':          pd.Timestamp(date).day_name(),
        'morning_move': round(morning_move, 1),
        'direction':    direction,
        'pnl_pips':     round(pnl_pips, 1),
        'win':          pnl_pips > 0,
    })

lmm_df = pd.DataFrame(lmm_rows)

print(f'Total days analysed: {len(lmm_df)}')
print(f'Overall win rate   : {lmm_df["win"].mean()*100:.1f}%')
print(f'Avg P&L (all days) : {lmm_df["pnl_pips"].mean():.1f} pips')
print()

# Win rate and P&L by min_move threshold
thresholds = range(0, 35, 5)
thresh_rows = []
for t in thresholds:
    subset = lmm_df[lmm_df['morning_move'].abs() > t]
    if len(subset) == 0:
        continue
    thresh_rows.append({
        'min_move_pips': t,
        'days_traded':   len(subset),
        'win_rate':      round(subset['win'].mean() * 100, 1),
        'avg_pnl':       round(subset['pnl_pips'].mean(), 1),
        'total_pnl':     round(subset['pnl_pips'].sum(), 0),
        'profit_factor': round(
            subset[subset['pnl_pips'] > 0]['pnl_pips'].sum() /
            abs(subset[subset['pnl_pips'] < 0]['pnl_pips'].sum())
            if subset['pnl_pips'].lt(0).any() else float('inf'), 2
        )
    })

thresh_df = pd.DataFrame(thresh_rows).set_index('min_move_pips')
print('Win rate and profit factor by morning move threshold:')
display(thresh_df)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes[0].plot(thresh_df.index, thresh_df['win_rate'], marker='o', color='steelblue')
axes[0].axhline(50, color='red', linestyle='--')
axes[0].set_title('LMM — Win Rate by Threshold')
axes[0].set_xlabel('Min Morning Move (pips)')
axes[0].set_ylabel('Win Rate (%)')

axes[1].plot(thresh_df.index, thresh_df['profit_factor'], marker='o', color='green')
axes[1].axhline(1, color='red', linestyle='--')
axes[1].set_title('LMM — Profit Factor by Threshold')
axes[1].set_xlabel('Min Morning Move (pips)')
axes[1].set_ylabel('Profit Factor')

axes[2].plot(thresh_df.index, thresh_df['days_traded'], marker='o', color='orange')
axes[2].set_title('LMM — Days Traded by Threshold')
axes[2].set_xlabel('Min Morning Move (pips)')
axes[2].set_ylabel('Trading Days')

plt.suptitle('London Morning Momentum — Signal Quality vs Threshold', fontsize=12)
plt.tight_layout()
plt.show()

## 10. Strategy 2: London Morning Momentum — Implementation

In [ ]:
class LondonMorningMomentumBase(Strategy):
    """
    At 12:45 UTC, if Close is more than min_move_pips above/below the 08:00 open → enter.
    Exit at exit_hour UTC. No SL/TP — time exit only.
    """
    exit_hour     = 17
    min_move_pips = 10

    def init(self):
        pass

    def next(self):
        bar_time = self.data.index[-1]

        if bar_time.hour == self.exit_hour and bar_time.minute == 0 and self.position:
            self.position.close()
            return

        if self.position:
            return

        if bar_time.hour == 12 and bar_time.minute == 45:
            c   = self.data.Close[-1]
            ref = self.data.Open_0800[-1]
            move = (c - ref) * 10_000   # pips
            if move > self.min_move_pips:
                self.buy()
            elif move < -self.min_move_pips:
                self.sell()


class LondonMorningMomentum(Strategy):
    """
    Same as LondonMorningMomentumBase but with stop loss and take profit.
    """
    exit_hour     = 17
    min_move_pips = 10
    sl_pips       = 20
    tp_pips       = 40

    def init(self):
        pass

    def next(self):
        bar_time = self.data.index[-1]

        if bar_time.hour == self.exit_hour and bar_time.minute == 0 and self.position:
            self.position.close()
            return

        if self.position:
            return

        if bar_time.hour == 12 and bar_time.minute == 45:
            c   = self.data.Close[-1]
            ref = self.data.Open_0800[-1]
            move = (c - ref) * 10_000   # pips
            pip = 0.0001
            if move > self.min_move_pips:
                self.buy(
                    sl=c - self.sl_pips * pip,
                    tp=c + self.tp_pips * pip
                )
            elif move < -self.min_move_pips:
                self.sell(
                    sl=c + self.sl_pips * pip,
                    tp=c - self.tp_pips * pip
                )


print('✓ Strategy classes defined')
print('  - LondonMorningMomentumBase : time exit only')
print('  - LondonMorningMomentum     : with SL/TP')

## 11. Backtest: London Morning Momentum + Optimisation

In [ ]:
# Base strategy (time exit)
bt_lmm_base = Backtest(df, LondonMorningMomentumBase, cash=INITIAL_CASH, commission=COMMISSION)
stats_lmm_base = bt_lmm_base.run()

print('=== LMM Base (time exit) ===')
print(f"Return          : {stats_lmm_base['Return [%]']:.2f}%")
print(f"Sharpe Ratio    : {stats_lmm_base['Sharpe Ratio']:.3f}")
print(f"Win Rate        : {stats_lmm_base['Win Rate [%]']:.1f}%")
print(f"Max Drawdown    : {stats_lmm_base['Max. Drawdown [%]']:.2f}%")
print(f"# Trades        : {stats_lmm_base['# Trades']}")
print()

# SL/TP strategy
bt_lmm_sltp = Backtest(df, LondonMorningMomentum, cash=INITIAL_CASH, commission=COMMISSION)
stats_lmm_sltp = bt_lmm_sltp.run()

print('=== LMM with SL/TP (default: 20/40 pips, min_move=10) ===')
print(f"Return          : {stats_lmm_sltp['Return [%]']:.2f}%")
print(f"Sharpe Ratio    : {stats_lmm_sltp['Sharpe Ratio']:.3f}")
print(f"Win Rate        : {stats_lmm_sltp['Win Rate [%]']:.1f}%")
print(f"Max Drawdown    : {stats_lmm_sltp['Max. Drawdown [%]']:.2f}%")
print(f"# Trades        : {stats_lmm_sltp['# Trades']}")

In [ ]:
# Optimise LMM SL/TP
print('Optimising LMM (this may take ~2 minutes)...')

bt_lmm_opt = Backtest(df, LondonMorningMomentum, cash=INITIAL_CASH, commission=COMMISSION)
stats_lmm_opt = bt_lmm_opt.optimize(
    min_move_pips=range(5, 31, 5),
    sl_pips=range(10, 41, 5),
    tp_pips=range(20, 81, 10),
    exit_hour=range(15, 20),
    maximize='Sharpe Ratio',
    constraint=lambda p: p.tp_pips > p.sl_pips,
    return_heatmap=False,
)

print('\n' + '=' * 60)
print('LMM OPTIMISED RESULTS')
print('=' * 60)
print(f"Return          : {stats_lmm_opt['Return [%]']:.2f}%")
print(f"Sharpe Ratio    : {stats_lmm_opt['Sharpe Ratio']:.3f}")
print(f"Win Rate        : {stats_lmm_opt['Win Rate [%]']:.1f}%")
print(f"Max Drawdown    : {stats_lmm_opt['Max. Drawdown [%]']:.2f}%")
print(f"# Trades        : {stats_lmm_opt['# Trades']}")
print(f"Best min_move   : {stats_lmm_opt._strategy.min_move_pips} pips")
print(f"Best SL         : {stats_lmm_opt._strategy.sl_pips} pips")
print(f"Best TP         : {stats_lmm_opt._strategy.tp_pips} pips")
print(f"Best exit       : {stats_lmm_opt._strategy.exit_hour}:00 UTC")

In [ ]:
# Equity curves — LMM
eq_lmm_base = stats_lmm_base['_equity_curve']
eq_lmm_sltp = stats_lmm_sltp['_equity_curve']
eq_lmm_opt  = stats_lmm_opt['_equity_curve']

fig = go.Figure()
fig.add_trace(go.Scatter(x=eq_lmm_base.index, y=eq_lmm_base['Equity'],
                         name='LMM Base (time exit)', line=dict(width=1.5)))
fig.add_trace(go.Scatter(x=eq_lmm_sltp.index, y=eq_lmm_sltp['Equity'],
                         name='LMM SL/TP Default', line=dict(width=1.5)))
fig.add_trace(go.Scatter(x=eq_lmm_opt.index, y=eq_lmm_opt['Equity'],
                         name='LMM Optimised', line=dict(width=2, dash='dot')))
fig.add_hline(y=INITIAL_CASH, line_dash='dash', line_color='gray', annotation_text='Initial capital')
fig.update_layout(
    title='London Morning Momentum — Equity Curves',
    xaxis_title='Date', yaxis_title='Equity ($)',
    height=450, hovermode='x unified'
)
fig.show()

## 12. Strategy Comparison

Side-by-side comparison of all four variants.

In [ ]:
METRICS = ['Return [%]', 'Sharpe Ratio', 'Win Rate [%]', 'Max. Drawdown [%]', '# Trades']

variants = {
    'ARB Base':      stats_arb_base,
    'ARB SL/TP':     stats_arb_sltp,
    'ARB Optimised': stats_arb_opt,
    'LMM Base':      stats_lmm_base,
    'LMM SL/TP':     stats_lmm_sltp,
    'LMM Optimised': stats_lmm_opt,
}

comparison = pd.DataFrame({
    name: {m: round(stats[m], 2) for m in METRICS}
    for name, stats in variants.items()
})

print('Strategy Comparison — GBP/USD M15 2025\n')
display(comparison)

# Combined equity curve chart
fig = go.Figure()
colors = ['blue', 'royalblue', 'darkblue', 'green', 'limegreen', 'darkgreen']
dashes = ['solid', 'dash', 'dot', 'solid', 'dash', 'dot']

for (name, stats), color, dash in zip(variants.items(), colors, dashes):
    eq = stats['_equity_curve']
    fig.add_trace(go.Scatter(
        x=eq.index, y=eq['Equity'],
        name=name, line=dict(color=color, width=1.5, dash=dash)
    ))

fig.add_hline(y=INITIAL_CASH, line_dash='dash', line_color='gray', annotation_text='Initial capital')
fig.update_layout(
    title='GBP/USD M15 2025 — All Strategy Equity Curves',
    xaxis_title='Date', yaxis_title='Equity ($)',
    height=500, hovermode='x unified'
)
fig.show()

## 13. Conclusions

### Strategy Summary

| Strategy | Return % | Sharpe | Win Rate | Max DD | # Trades |
|----------|----------|--------|----------|--------|----------|
| ARB Base | *(fill)* | *(fill)* | *(fill)* | *(fill)* | *(fill)* |
| ARB SL/TP | *(fill)* | *(fill)* | *(fill)* | *(fill)* | *(fill)* |
| ARB Optimised | *(fill)* | *(fill)* | *(fill)* | *(fill)* | *(fill)* |
| LMM Base | *(fill)* | *(fill)* | *(fill)* | *(fill)* | *(fill)* |
| LMM SL/TP | *(fill)* | *(fill)* | *(fill)* | *(fill)* | *(fill)* |
| LMM Optimised | *(fill)* | *(fill)* | *(fill)* | *(fill)* | *(fill)* |

### Key Findings

| Question | Answer |
|----------|--------|
| Which strategy has the better Sharpe Ratio? | *(fill after running)* |
| Does adding SL/TP improve or hurt vs time exit? | *(fill after running)* |
| What is the optimal ARB SL/TP combination? | *(fill after running)* |
| What is the optimal LMM min_move threshold? | *(fill after running)* |
| Do both strategies complement each other? | *(fill after running)* |

### Strengths of These Strategies
- **ARB**: Uses structural session boundaries, not arbitrary indicator levels
- **LMM**: Filters for trending days only — skips flat/choppy sessions
- Both: Fully intraday, zero overnight exposure, no CFD financing risk
- Both: One trade per day, simple position management

### Weaknesses and Risks
- In-sample only — **must validate on out-of-sample data** (e.g., 2024 data)
- Asian range can be very tight on illiquid days → false breakouts
- LMM momentum may reverse sharply on news events (e.g., UK CPI, US NFP)
- Optimised parameters may be overfit to 2025 — treat with caution

### Recommended Next Steps
1. **Out-of-sample test**: run both on 2024 GBP/USD data
2. **News filter**: exclude days with high-impact UK/US economic releases
3. **Asian range size filter**: only trade ARB when Asian range > N pips (avoid thin ranges)
4. **Portfolio test**: combine ARB + LMM on same capital, check correlation of losses
5. **Walk-forward analysis**: rolling 3-month train / 1-month test windows

---
**Reminder**: In-sample backtest results are optimistic. Past performance does not guarantee future results.